In [0]:
import plotly.graph_objects as go
import numpy as np
import pickle

In [0]:
# get data
# path
RESULTS_PATH = ''
model_type = 'distilgpt2'
data = 'wb' # 'wg' or 'wb'
data_type = 'head' # or 'layer'
if model_type == 'distilgpt2':
    head_size = (6,12)
elif model_type == 'gpt2':
    head_size = (12,12)
elif model_type == 'gpt2-medium':
    head_size = (24,12)

greedy = pickle.load( open( RESULTS_PATH + 'greedy_' + model_type +  '_' + data + '_full.pickle', "rb" )) 
obj_list_gr = greedy['val']

In [17]:
num_steps

72

In [25]:
# Create figure
fig = go.Figure()

if data_type == 'layer':
  num_steps = head_size[0]*head_size[1]
  y = np.zeros(head_size[0])
  x = list(range(head_size[0]))
  xlim = head_size[0]
  ylim = head_size[1]
else:
  num_steps = head_size[0]*head_size[1]
  y = np.zeros(head_size[1])
  x = list(range(head_size[1]))
  xlim = head_size[1]
  ylim = head_size[0]

# Add traces, one for each slider step
for step in range(num_steps):
    if data_type == 'layer':
      y[greedy['head'][step][0]] += 1
    else:
      y[greedy['head'][step][1]] += 1
    data = go.Bar(x=x, y=y, 
                  opacity=obj_list_gr[step]/np.max(obj_list_gr), 
                  name='Val: ' + str(round(obj_list_gr[step], 3)))

    fig.add_trace(data)

fig['layout']['xaxis'].update(title='', range=[-1, xlim], autorange=False)
fig['layout']['yaxis'].update(title='', range=[0, ylim], autorange=False)

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Greedy: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()